In [7]:
import aocd
data = aocd.get_data(day=1, year=2023)
T = data.split()

trans = {'one':'o1e', 'two':'t2o', 'three':'3e', 'four':'4',
         'five':'5e', 'six':'6', 'seven':'7n', 'eight':'e8t',
         'nine':'n9e'}

def tr(x):
    for s in trans: x = x.replace(s, trans[s])
    return x

def cal(x):
    l = [int(c) for c in x if c.isdigit()]
    return 10*l[0]+l[-1]

print("Part1:", sum(map(cal, T)), "Part2:",sum(map(cal, map(tr,T))))

Part1: 54388 Part2: 53515


In [23]:
import aocd
from collections import defaultdict
data = aocd.get_data(day=2, year=2023)

part1 = part2 = 0

for i, g in enumerate(data.split("\n")):
    _,l = g.split(": ")
    p1 = i+1
    maxes = defaultdict(int)
    for u in l.split("; "):
        for t in u.split(", "):
            n, c = t.split(" ")
            n = int(n)
            if not((c == "red" and n <= 12) 
                or (c == "green" and n <= 13) 
                or (c == "blue" and n <= 14)): p1 = 0
            maxes[c] = max(maxes[c], n)
    part1 += p1
    p2 = 1
    for k in maxes: p2 *= maxes[k]
    part2 += p2
print(part1,part2)

2795 75561
